In [ ]:
import tensorboard

In [ ]:
%load_ext autoreload
%autoreload 2

import logging
import numpy as np
import sys
import pandas as pd
import pickle as pkl

from sklearn.preprocessing import minmax_scale
import tensorflow as tf
tf.get_logger().setLevel('ERROR') # only show error messages

from recommenders.utils.python_utils import binarize
from recommenders.utils.timer import Timer
from recommenders.models.ncf.ncf_singlenode import NCF
from recommenders.models.sar import SAR
from recommenders.models.ncf.dataset import Dataset as NCFDataset
from recommenders.utils.notebook_utils import is_jupyter
from recommenders.datasets.python_splitters import python_chrono_split, python_stratified_split
from recommenders.evaluation.python_evaluation import (rmse, mae, rsquared, exp_var, map_at_k, ndcg_at_k, precision_at_k, 
                                                     recall_at_k, get_top_k_items,logloss)

from sklearn.ensemble import GradientBoostingRegressor as GBRT
from sklearn.model_selection import GridSearchCV

from surprise import Dataset, Reader, SVD

SEED = 42
MODEL_DIR = './models/'
TRAIN_MODELS = False

Load Dataset

In [ ]:
df = pd.read_csv('ml-latest-small/ratings.csv', dtype = {"userId": int, "movieId": int, "rating": float, "timestamp": int,})
df.columns = ['userID', 'itemID', 'rating', 'timestamp']

In [ ]:
t = df[['userID', 'rating']]
t['rating'].plot.hist(bins = 4)

Evaluation metrics

In [ ]:
def get_errors(data, pred):
    eval_rmse = rmse(data, pred, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction')
    eval_mae = mae(data, pred, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction')
    return eval_rmse, eval_mae

def get_rank_metrics(data, pred, top_k=5):
    eval_ndcg = ndcg_at_k(data, pred, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction', k=top_k)
    eval_precision = precision_at_k(data, pred, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction', k=top_k)
    eval_recall = recall_at_k(data, pred, col_user='userID', col_item='itemID', col_rating='rating', col_prediction='prediction', k=top_k, relevancy_method='top_k')
    eval_f1 = (2 * eval_precision * eval_recall) / (eval_precision + eval_recall)
    return eval_ndcg, eval_precision, eval_recall, eval_f1

Create train test data for SAR

In [ ]:
train, test = python_stratified_split(df, ratio=0.8, col_user='userID', col_item='itemID', seed=SEED)

test = test[test["userID"].isin(train["userID"].unique())]
test = test[test["itemID"].isin(train["itemID"].unique())]

train_file = "./train.csv"
test_file = "./test.csv"
train.to_csv(train_file, index=False)
test.to_csv(test_file, index=False)

SAR implementation

In [ ]:
if TRAIN_MODELS:
    sar_model = SAR(
        col_user="userID",
        col_item="itemID",
        col_rating="rating",
        col_timestamp="timestamp",
        similarity_type="jaccard", 
        time_decay_coefficient=30, 
        timedecay_formula=True,
        normalize=True
    )
    with Timer() as train_time:
        sar_model.fit(train)
    print("Took {} seconds for training.".format(train_time.interval))

    file = open(MODEL_DIR+'sar', 'wb')
    pkl.dump(sar_model, file)
    file.close()
    
sar_model = pkl.load(open(MODEL_DIR+'sar', 'rb'))

SAR error analysis

In [ ]:
# with Timer() as test_time:
#     preds = sar_model.predict(train)
# print("Took {} seconds for prediction.".format(test_time.interval))

# top_k = 10
# eval_rmse, eval_mae = get_errors(train, preds)
# top_k_rec = sar_model.recommend_k_items(train, top_k=top_k, remove_seen = False)
# eval_ndcg, eval_precision, eval_recall = get_rank_metrics(train, top_k_rec, top_k)
# print("Model:\t",
#       "\nTop K:\t%d" % top_k,
#       "\nNDCG:\t%f" % eval_ndcg,
#       "\nPrecision@K:\t%f" % eval_precision,
#       "\nRecall@K:\t%f" % eval_recall,
#       "\nRMSE:\t%f" % eval_rmse,
#       "\nMAE:\t%f" % eval_mae,)

In [ ]:
with Timer() as test_time:
    preds = sar_model.predict(test)
print("Took {} seconds for prediction.".format(test_time.interval))

top_k = 10
eval_rmse, eval_mae = get_errors(test, preds)
top_k_rec = sar_model.recommend_k_items(test, top_k=top_k, remove_seen = True)
eval_ndcg, eval_precision, eval_recall, eval_f1 = get_rank_metrics(test, top_k_rec, top_k)
print("Model:\t",
      "\nTop K:\t%d" % top_k,
      "\nNDCG:\t%f" % eval_ndcg,
      "\nPrecision@K:\t%f" % eval_precision,
      "\nRecall@K:\t%f" % eval_recall,
      "\nRMSE:\t%f" % eval_rmse,
      "\nMAE:\t%f" % eval_mae,
      "\nF1-score:\t%f" % eval_f1,)

In [ ]:
svd_model = SVD(n_epochs=25, lr_all=0.01, reg_all=0.4)

In [ ]:
reader = Reader(rating_scale=(0,5))

trainset = Dataset.load_from_df(train[['userID', 'itemID', 'rating']], reader)
train_list = trainset.construct_testset(trainset.raw_ratings)
trainset = trainset.construct_trainset(trainset.raw_ratings)

testset = Dataset.load_from_df(test[['userID', 'itemID', 'rating']], reader)
testset = testset.construct_testset(testset.raw_ratings)

In [ ]:
svd_model.fit(trainset)

In [ ]:
svd_train_preds = svd_model.test(train_list)
svd_train_pred = pd.DataFrame(svd_train_preds, columns=['userID', 'itemID', 'rating', 'svd_prediction', 'details'])
svd_train_pred = svd_train_pred.drop(columns=['details', 'rating'])

In [ ]:
svd_test_preds = svd_model.test(testset)
svd_test_pred = pd.DataFrame(svd_test_preds, columns=['userID', 'itemID', 'rating', 'prediction', 'details'])
top_k = 10
eval_rmse, eval_mae = get_errors(svd_test_pred[['userID', 'itemID', 'rating']], svd_test_pred[['userID', 'itemID', 'prediction']])
svd_test_top_k = svd_test_pred.sort_values(by=['prediction'], ascending=False).groupby('userID').head(5)
eval_ndcg, eval_precision, eval_recall, eval_f1 = get_rank_metrics(svd_test_pred[['userID', 'itemID', 'rating']], svd_test_top_k[['userID', 'itemID', 'prediction']], top_k)
print("Model:\t",
      "\nTop K:\t%d" % top_k,
      "\nNDCG:\t%f" % eval_ndcg,
      "\nPrecision@K:\t%f" % eval_precision,
      "\nRecall@K:\t%f" % eval_recall,
      "\nRMSE:\t%f" % eval_rmse,
      "\nMAE:\t%f" % eval_mae,
      "\nF1-score:\t%f" % eval_f1,)
svd_test_pred = svd_test_pred.rename({'prediction': 'svd_prediction'}, axis='columns')
svd_test_pred = svd_test_pred.drop(columns=['details', 'rating'])

Popularity feature

In [ ]:
popularity = pd.DataFrame(df['itemID'].value_counts())
popularity = popularity.reset_index()
popularity.columns = ['itemID', 'popularity']

GBRT data prep

In [ ]:
sar_train = sar_model.predict(train)
sar_train = sar_train.rename({'prediction': 'sar_prediction'}, axis='columns')

gbrt_train = train.merge(popularity, on='itemID', how='left').merge(sar_train, on=['itemID', 'userID'], how='left').merge(svd_train_pred, on=['itemID', 'userID'], how='left')
gbrt_train = gbrt_train.drop(columns=['timestamp'], axis=1)

In [ ]:
sar_test = sar_model.predict(test)
sar_test = sar_test.rename({'prediction': 'sar_prediction'}, axis='columns')

gbrt_test = test.merge(popularity, on='itemID', how='left').merge(sar_test, on=['itemID', 'userID'], how='left').merge(svd_test_pred, on=['itemID', 'userID'], how='left')
gbrt_test = gbrt_test.drop(columns=['timestamp'], axis=1)

Hypertuning GBRT (hybrid model)

In [ ]:
if TRAIN_MODELS:
    n_estimators = [100]
    learn_rates = [0.05]
    max_depths = [5]

    param_grid = {'n_estimators': n_estimators,
                'learning_rate': learn_rates,
                'max_depth': max_depths,}

    grid_search = GridSearchCV(GBRT(loss='huber'),param_grid, cv=5, return_train_score=True)
    grid_search.fit(gbrt_train.drop(columns=['userID', 'itemID', 'rating'], axis=1), gbrt_train['rating'])

    file = open(MODEL_DIR+'gbrt', 'wb')
    pkl.dump(grid_search, file)
    file.close()
    
grid_search = pkl.load(open(MODEL_DIR+'gbrt', 'rb'))
gbrt = grid_search.best_estimator_

In [ ]:

# pred = gbrt.predict(gbrt_train.drop(columns=['userID', 'itemID', 'rating'], axis=1))
# gbrt_train['prediction'] = pred
# top_k = 10
# eval_rmse, eval_mae = get_errors(gbrt_train[['userID', 'itemID', 'rating']], gbrt_train[['userID', 'itemID', 'prediction']])
# gbrt_train_top_k = gbrt_train.sort_values(by=['prediction'], ascending=False).groupby('userID').head(5)
# eval_ndcg, eval_precision, eval_recall = get_rank_metrics(gbrt_train[['userID', 'itemID', 'rating']], gbrt_train_top_k[['userID', 'itemID', 'prediction']], top_k)
# print("Model:\t",
#       "\nTop K:\t%d" % top_k,
#       "\nNDCG:\t%f" % eval_ndcg,
#       "\nPrecision@K:\t%f" % eval_precision,
#       "\nRecall@K:\t%f" % eval_recall,
#       "\nRMSE:\t%f" % eval_rmse,
#       "\nMAE:\t%f" % eval_mae,)

In [ ]:
pred = gbrt.predict(gbrt_test.drop(columns=['userID', 'itemID', 'rating'], axis=1))
gbrt_test['prediction'] = pred
top_k = 10
eval_rmse, eval_mae = get_errors(gbrt_test[['userID', 'itemID', 'rating']], gbrt_test[['userID', 'itemID', 'prediction']])
gbrt_test_top_k = gbrt_test.sort_values(by=['prediction'], ascending=False).groupby('userID').head(5)
eval_ndcg, eval_precision, eval_recall, eval_f1 = get_rank_metrics(gbrt_test[['userID', 'itemID', 'rating']], gbrt_test_top_k[['userID', 'itemID', 'prediction']], top_k)
print("Model:\t",
      "\nTop K:\t%d" % top_k,
      "\nNDCG:\t%f" % eval_ndcg,
      "\nPrecision@K:\t%f" % eval_precision,
      "\nRecall@K:\t%f" % eval_recall,
      "\nRMSE:\t%f" % eval_rmse,
      "\nMAE:\t%f" % eval_mae,
      "\nF1-score:\t%f" % eval_f1,)

In [ ]:
gbrt_test_top_k[gbrt_test_top_k['userID'] == 1]

In [ ]:
for imp in gbrt.feature_importances_:
    print(imp)
print(gbrt_train.drop(['userID', 'itemID', 'rating'], axis=1).columns)